# Create Agents to Research, Write and Edit a Letter of Correspondence

## Setting a Goal

Write a letter of correspondence on an issue from sender to recipient.

In [1]:
# Install required libraries
!pip install -q 'crewai[tools]' langchain-groq duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 665.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import modules
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
from google.colab import userdata

In [3]:
# Access Colab secret keys
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [4]:
# Initialize ChatGroq class with a model
llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="llama3-8b-8192"
)

In [5]:
# Use DuckDuckGo Search component
@tool('DuckDuckGoSearch')
def search(query: str):
    """
    Search the web using DuckDuckGo with an input query.
    """
    return DuckDuckGoSearchRun().run(query)

## Creating Agents

### Agent: Researcher

In [6]:
researcher = Agent(
    role="Senior Researcher",
    goal="Collect foundational research on {issue}, {sender}, and {recipient}.",
    backstory="An expert researcher who provides valuable insights and recommendations that inform decision making.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Analyst

In [7]:
analyst = Agent(
    role="Senior Analyst",
    goal="Analyze the research findings about {issue}, {sender}, and {recipient}.",
    backstory="An expert analyst who identifies patterns and trends in the research findings.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

### Agent: Writer

In [8]:
writer = Agent(
    role="Senior Writer",
    goal="Write a professional letter of correspondence on {issue} from {sender} to {recipient}.",
    backstory="You are an expert at writing letters that require careful consideration and attention to detail.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [9]:
editor = Agent(
    role="Senior Editor",
    goal="Edit a professional letter of correspondence on {issue} from {sender} to {recipient}.",
    backstory="An expert editor who ensures the letter of correspondence provides balanced perspectives on opinions and assertions.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

### Task: Research

In [10]:
research_issue = Task(
    description="Research information on {issue}.",
    expected_output="A bullet point list summarizing the key findings, analysis, and recommendations.",
    agent=researcher,
    async_execution=True,
    tools=[search]
)

In [11]:
research_sender = Task(
    description="Research {sender}'s statements on {issue}.",
    expected_output="A bullet point list summarizing the key findings, analysis, and recommendations.",
    agent=researcher,
    async_execution=True,
    tools=[search]
)

In [12]:
research_recipient = Task(
    description="Research {recipient}'s statements on {issue}.",
    expected_output="A bullet point list summarizing the key findings, analysis, and recommendations.",
    agent=researcher,
    async_execution=True,
    tools=[search]
)

### Task: Analyze

In [13]:
analyze = Task(
    description="Analyze the research findings about {issue}, {sender}, and {recipient}.",
    expected_output="A comprehensive outline for a letter of correspondence on {issue} from {sender} to {recipient}.",
    agent=analyst,
    context=[research_issue, research_sender, research_recipient]
)

### Task: Write

In [14]:
write = Task(
    description=(
        "1. Write a clear and concise letter of correspondence on {issue} from {sender} to {recipient}.\n"
        "2. Use a formal salutation to greet {recipient} with the proper title and honorifics.\n"
        "3. Use a formal closing, such as 'Sincerely' or 'Yours faithfully' to end the letter.\n"
        "4. Include {sender}'s full name, spaced 3 lines after the closing."
    ),
    expected_output="A professional letter of correspondence in markdown format, including the following sections: formal greeting, purpose, background, proposal, solutions, alternatives, and formal closing.",
    agent=writer,
    context=[analyze]
)

### Task: Edit

In [15]:
edit = Task(
    description=(
        "1. Edit a letter of correspondence on {issue} from {sender} to {recipient}.\n"
        "2. Ensure the letter is accurate and relevant."
    ),
    expected_output="An edited professional letter of correspondence in markdown format, including the following sections: formal greeting, purpose, background, proposal, solutions, alternatives, and formal closing.",
    agent=editor,
    context=[write],
    output_file="letter.md"
)

## Creating the Crew

In [16]:
crew = Crew(
    agents=[researcher, analyst, writer, editor],
    tasks=[research_issue, research_sender, research_recipient, analyze, write, edit],
    process=Process.sequential,
    verbose=True
)

## Running the Crew

In [17]:
issue = "diss tracks"
sender = "Drake"
recipient = "Kendrick Lamar"

In [18]:
inputs = {
    "issue": issue,
    "sender": sender,
    "recipient": recipient
}

In [19]:
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Researcher
 [INFO]: == Starting Task: Research information on diss tracks.
 [DEBUG]: == [Senior Researcher] Task output: 


 [DEBUG]: == Working Agent: Senior Researcher
 [INFO]: == Starting Task: Research Drake's statements on diss tracks.
 [DEBUG]: == [Senior Researcher] Task output: 


 [DEBUG]: == Working Agent: Senior Researcher
 [INFO]: == Starting Task: Research Kendrick Lamar's statements on diss tracks.


> Entering new CrewAgentExecutor chain... [DEBUG]: == [Senior Researcher] Task output: 


 [DEBUG]: == Working Agent: Senior Analyst
 [INFO]: == Starting Task: Analyze the research findings about diss tracks, Drake, and Kendrick Lamar.


> Entering new CrewAgentExecutor chain...



> Entering new CrewAgentExecutor chain...
Action: DuckDuckGoSearch
Action Input: { "query": "'Kendrick Lamar diss tracks'" }Thought: I should start by researching diss tracks in general, and then focus on diss tracks related to Drake and Kendrick Lamar.

Action: D

In [20]:
from IPython.display import Markdown
Markdown(result)

**[Your Honorifics]**
**Kendrick Lamar**
**[His Address]**
**[City, State, ZIP]**
**[Email Address]**
**[Date]**

Dear Mr. Lamar,

**Purpose**
I am writing to provide a balanced analysis of the diss tracks exchanged between you and Drake, with the aim of understanding the motivations behind these tracks and their impact on the music industry.

**Background**
The feud between the two artists began in 2015 with the release of Drake's "Back to Back," a response to your "Control." The feud escalated with the release of your "The Heart Part 4" in 2016, which many saw as a response to Drake's "Champagne Moments." The feud continued with the release of Drake's "Savage" in 2024, which was seen as a response to your "Euphoria."

**Proposal**
I propose that the diss tracks exchanged between the two artists have been numerous and have sparked a lot of debate in the music industry. Some of the notable diss tracks include:

* Drake's "Back to Back" (2015) - a response to your "Control"
* Your "The Heart Part 4" (2016) - a response to Drake's "Champagne Moments"
* Drake's "Savage" (2024) - a response to your "Euphoria"
* Your "6:16 in LA" (2024) - a response to Drake's "Savage"

**Solutions**
To move forward, I suggest that both artists consider the impact their diss tracks have on their fans and the music industry as a whole. It is essential to maintain a level of professionalism and respect in their interactions, even in the midst of a feud.

**Alternatives**
In addition to diss tracks, I propose that both artists consider alternative forms of expression, such as collaborative projects or public statements, to address any issues or disagreements they may have.

**Formal Closing**
In conclusion, the feud between Drake and Kendrick Lamar has been a topic of discussion in the music industry for some time. The diss tracks exchanged between the two artists have been numerous and have sparked a lot of debate. I hope that this analysis has provided a balanced perspective on the motivations behind the diss tracks and their impact on the artists involved.

Sincerely,

Aubrey Drake Graham